In [1]:
from bs4 import BeautifulSoup
import requests
# Parsing and creating xml data
from lxml import etree as et

# Store data as a csv file written out
from csv import writer

# In general to use with timing our function calls to Indeed
import time

# Assist with creating incremental timing for our scraping to seem more human
from time import sleep

# Dataframe stuff
import pandas as pd
import os
import re
import numpy as np
import time
import unidecode
from tqdm import tqdm
import csv

# Random integer for more realistic timing for clicks, buttons and searches during scraping
import random
from random import randint

# Multi Threading
import threading

# Threading:
from concurrent.futures import ThreadPoolExecutor, wait
import math
import mysql.connector
from datetime import date
import json

In [2]:
import selenium

# Check version I am running
selenium.__version__

'4.31.1'

In [3]:
import undetected_chromedriver as uc


In [4]:
from selenium import webdriver

# Starting/Stopping Driver: can specify ports or location but not remote access
from selenium.webdriver.chrome.service import Service as ChromeService

# Manages Binaries needed for WebDriver without installing anything directly
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.action_chains import ActionChains

In [5]:
# Allows searchs similar to beautiful soup: find_all
from selenium.webdriver.common.by import By

# Try to establish wait times for the page to load
from selenium.webdriver.support.ui import WebDriverWait

# Wait for specific condition based on defined task: web elements, boolean are examples
from selenium.webdriver.support import expected_conditions as EC

# Used for keyboard movements, up/down, left/right,delete, etc
from selenium.webdriver.common.keys import Keys

# Locate elements on page and throw error if they do not exist
from selenium.common.exceptions import NoSuchElementException

In [6]:
response = requests.get(
  url='https://headers.scrapeops.io/v1/browser-headers',
  params={
      'api_key': 'cca4ced0-490d-41a0-b258-46f2ad7e74b3',
      'num_results': '100'}
)
header_browser_list = response.json()
print(header_browser_list['result'][0])

{'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"', 'sec-ch-ua-mobile': '?0', 'sec-ch-ua-platform': '"Linux"', 'upgrade-insecure-requests': '1', 'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36', 'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7', 'sec-fetch-site': 'same-site', 'sec-fetch-mode': 'navigate', 'sec-fetch-user': '?1', 'sec-fetch-dest': 'document', 'accept-encoding': 'gzip, deflate, br, zstd', 'accept-language': 'en-US'}


In [7]:
response = requests.get(
  url='https://headers.scrapeops.io/v1/user-agents',
  params={
      'api_key': 'cca4ced0-490d-41a0-b258-46f2ad7e74b3',
      'num_results': '100'}
)
user_agent_list = response.json()
print(user_agent_list['result'][0])

Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Mobile Safari/537.36


In [8]:
random_index_user_agent = randint(0, len(user_agent_list)-1)
random_index_header_browser = randint(0, len(header_browser_list)-1)
user_agent_random = user_agent_list['result'][random_index_user_agent]
header_browser_random = header_browser_list['result'][random_index_header_browser]

In [9]:
def create_option():
    random_index_user_agent = randint(0, len(user_agent_list)-1)
    random_index_header_browser = randint(0, len(header_browser_list)-1)
    user_agent_random = user_agent_list['result'][random_index_user_agent]
    header_browser_random = header_browser_list['result'][random_index_header_browser]
    # Allows you to cusotmize: ingonito mode, maximize window size, headless browser, disable certain features, etc
    option = uc.ChromeOptions()

    # Going undercover:
    #option.add_argument("--incognito")


    # # Consider this if the application works and you know how it works for speed ups and rendering!

    # option.add_argument('--headless=chrome')
    user_agent = user_agent_random
    option.add_argument(f"user-agent={user_agent}")

    # Thêm header vào Options
    headers = header_browser_random

    for key, value in headers.items():
        option.add_argument(f"--header={key}:{value}")
    
    return option

In [10]:
driver = uc.Chrome(service=ChromeService(ChromeDriverManager().install()),options=create_option())
#driver.implicitly_wait(10)  # Set the implicit wait time to 10 seconds
#driver.set_page_load_timeout(30)
driver.get("https://www.tripadvisor.com")
driver.maximize_window()


In [11]:
# Tìm và nhấn vào tìm kiếm nhà hàng với XPath
attraction_ele = driver.find_element(By.XPATH, "//a[@href='/Attractions']")
attraction_ele.click()

# Nhập thành phố
country = 'Cambodia'
place = 'Banteay Meanchey'
time.sleep(random.uniform(0.8, 2))  # Thời gian trễ ngẫu nhiên

# Tìm thanh search
search_bar_location = driver.find_element(By.XPATH, "/html/body/div[1]/main/div[4]/div/div/div[2]/div/form/div[1]/div/input")

# Thêm thời gian trễ ngẫu nhiên giữa các hành động
time.sleep(random.uniform(0.8, 2))
search_bar_location.clear()

time.sleep(random.uniform(0.8, 2))
search_bar_location.send_keys(f"{place}")

# Di chuyển chuột đến thanh tìm kiếm để giả lập người dùng thật
actions = ActionChains(driver)
actions.move_to_element(search_bar_location).perform()
time.sleep(random.uniform(0.8, 2))

# Tìm liên kết "Hotels" và bấm vào
restaurant_link = driver.find_element(By.XPATH, "//div[@class='yOZMu u j']//a[1]")
time.sleep(random.uniform(2, 4))

# Bấm vào link "Hotels"
restaurant_link.click()
time.sleep(random.uniform(2, 4))




In [12]:
# Hàm chuyển đổi URL để thêm '-oa0-' vào giữa
def convert_tripadvisor_url(url):
    parts = url.split('-Activities-')
    if len(parts) != 2:
        return None  # Trả về None nếu URL không hợp lệ
    return f"{parts[0]}-Activities-oa0-{parts[1]}"

# Lấy URL hiện tại
current_url = driver.current_url

# Chuyển đổi URL với hàm mới
new_url = convert_tripadvisor_url(current_url)
if new_url:
    print(f"Chuyển sang URL mới: {new_url}")
    driver.get(new_url)  # Mở URL đã chuyển đổi

time.sleep(random.uniform(2, 4))  # Chờ để xem nội dung trang mới

Chuyển sang URL mới: https://www.tripadvisor.com/Attractions-g1074270-Activities-oa0-Banteay_Meanchey_Province.html


In [13]:
def getURL():
    # Parse HTML từ trình duyệt
    page_source = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Kiểm tra lại class và tìm các thẻ <a> có thuộc tính href
    attractions = page_source.find_all('a', href=True, target='_blank')
    all_attraction_url = []

    for attraction in attractions:
        attraction_link = attraction.get('href')
        # Kiểm tra nếu URL hợp lệ và không chứa '#'
        if attraction_link and '#' not in attraction_link:
            full_url = 'https://www.tripadvisor.com' + attraction_link
            if full_url not in all_attraction_url:
                all_attraction_url.append(full_url)

    return all_attraction_url

# Gọi hàm và in kết quả
getURL()



['https://www.tripadvisor.com/Attraction_Review-g7023045-d1462656-Reviews-Banteay_Chhmar_Temple-Banteay_Chhmar_Banteay_Meanchey_Province.html',
 'https://www.tripadvisor.com/ClientLink?value=a0dJXy9BdHRyYWN0aW9uX1Byb2R1Y3RzLWcxMDc0MjcwLWQxNDYyNjU2LUJhbnRlYXlfQ2hobWFyX1RlbXBsZS1CYW50ZWF5X01lYW5jaGV5X1Byb3ZpbmNlLmh0bWxfVnE1',
 'https://www.tripadvisor.com/Attraction_Review-g1075851-d8353424-Reviews-Organikh_Project-Sisophon_Banteay_Meanchey_Province.html',
 'https://www.tripadvisor.com/Attraction_Review-g1544600-d7981149-Reviews-Prasat_Prei_Monti-Roluos_Banteay_Meanchey_Province.html',
 'https://www.tripadvisor.com/ClientLink?value=SkhFXy9Qcm9maWxlL0t1cnR6X0Jlbl96U2w%3D',
 'https://www.tripadvisor.com/ShowUserReviews-g1544600-d7981149-r688564559-Prasat_Prei_Monti-Roluos_Banteay_Meanchey_Province.html',
 'https://www.tripadvisor.com/ClientLink?value=U2tsXy9BdHRyYWN0aW9uX1Byb2R1Y3RzLWcxMDc0MjcwLWQ3OTgxMTQ5LVByYXNhdF9QcmVpX01vbnRpLUJhbnRlYXlfTWVhbmNoZXlfUHJvdmluY2UuaHRtbF9xMUU%3D',
 'https:

In [14]:
# Hàm lấy URL của tất cả các trang
URLs_all_page = []

def getUrlOnPages():
    input_page = 1000

    for page in range(input_page):
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break  
            last_height = new_height

        URLs_one_page = getURL()  
        # URLs_all_page.extend(URLs_one_page)  # Thêm URL của trang hiện tại vào danh sách tổng
        
        
        # Kiểm tra và chỉ thêm những URL chưa có trong danh sách
        for url in URLs_one_page:
            if 'Attraction_Review' in url and url not in URLs_all_page:
                URLs_all_page.append(url)
                
        try:
            next_button = driver.find_element(By.XPATH, "//a[@aria-label='Next page']")
            next_page_url = next_button.get_attribute('href')  
            driver.get(next_page_url)  
            time.sleep(random.uniform(2, 4))
        except:
            print(f"Không tìm thấy nút 'next' ở trang {page + 1}. Dừng scrape.")
            break  

    return URLs_all_page

# Gọi hàm
getUrlOnPages()


Không tìm thấy nút 'next' ở trang 1. Dừng scrape.


['https://www.tripadvisor.com/Attraction_Review-g7023045-d1462656-Reviews-Banteay_Chhmar_Temple-Banteay_Chhmar_Banteay_Meanchey_Province.html',
 'https://www.tripadvisor.com/Attraction_Review-g1075851-d8353424-Reviews-Organikh_Project-Sisophon_Banteay_Meanchey_Province.html',
 'https://www.tripadvisor.com/Attraction_Review-g1544600-d7981149-Reviews-Prasat_Prei_Monti-Roluos_Banteay_Meanchey_Province.html',
 'https://www.tripadvisor.com/Attraction_Review-g674568-d28655733-Reviews-Istana_Casino-Poipet_Banteay_Meanchey_Province.html',
 'https://www.tripadvisor.com/Attraction_Review-g1075851-d19274971-Reviews-Ang_Trapeng_Thmor_Reserve-Sisophon_Banteay_Meanchey_Province.html',
 'https://www.tripadvisor.com/Attraction_Review-g7023045-d32899208-Reviews-Soieries_Du_Mekong_Et_Workshop-Banteay_Chhmar_Banteay_Meanchey_Province.html',
 'https://www.tripadvisor.com/Attraction_Review-g674568-d27157701-Reviews-Vicci_Beauty_Bar_Poipet-Poipet_Banteay_Meanchey_Province.html',
 'https://www.tripadvisor.co

In [15]:
len(URLs_all_page)

10

In [16]:
# Kiểm tra xem file đã có dữ liệu chưa (để tránh ghi lại tiêu đề)
file_exists = os.path.isfile("Attraction_Data/Attraction_Data_SEA.csv")

with open(f"Attraction_Data/Attraction_Data_SEA.csv", 'a', newline='', encoding='utf-8-sig') as file_output:
    # Thêm các tiêu đề vào đây
    headers = [
        'Attraction ID', 'Country', 'Tourist Attraction', 'Name', 'Address', 'Website', 'Phone Number', 
        'Category', 'Ranking', 'Opening Hour', 'Description', 'Duration', 'Price', 'Email', 'Rating', 'Review Count', 
        'Excellent Reviews', 'Very Good Reviews', 'Average Reviews', 'Poor Reviews', 'Terrible Reviews', 
        'Near Restaurants', 'Near Attractions', "URL", 'Image URL']
    
    writer = csv.DictWriter(file_output, delimiter=",", lineterminator="\n", fieldnames=headers)

    # Ghi tiêu đề chỉ khi file trống (chưa có dữ liệu)
    if not file_exists:
        writer.writeheader()
        
    # Khởi tạo biến đếm cho Attraction ID
    attraction_id_counter = 72811

    # Thiết lập thanh tiến độ
    with tqdm(total=len(URLs_all_page), desc="Crawling attraction data", dynamic_ncols=True, unit="attraction", leave=False) as pbar:
        for attraction_url in URLs_all_page:
            driver.get(attraction_url)
            time.sleep(2) 
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, "html.parser")
        

            # Lấy thông tin tên 
            name = re.sub(r'\s+', ' ', driver.find_element(By.CSS_SELECTOR, '.biGQs._P.fiohW.eIegw').text).strip()
                        
            #time.sleep(random.uniform(0.2, 0.28))
            
            # Lấy thông tin category
            category = [span.get_text() for span in soup.find_all("span", class_="eojVo")]
            if category:
                category = category 
            else:
                category = np.nan
            #time.sleep(random.uniform(0.2, 0.28))

            # Lấy số lượng đánh giá
            review_count_tag = soup.find('span', {'data-automation': 'reviewCount'})
            if review_count_tag:
                review_count = review_count_tag.get_text(strip=True).split()[0] 
            else:
                review_count = np.nan
            #time.sleep(random.uniform(0.2, 0.28))

            # Lấy số sao 
            rating_tag = soup.find('svg', {'data-automation': 'reviewBubbles'})
            if rating_tag:
                title_tag = rating_tag.find('title')
                rating_value = title_tag.get_text(strip=True).split()[0] if title_tag else np.nan
            else:
                rating_value = np.nan
            #time.sleep(random.uniform(0.2, 0.28))

            # Tìm thông tin mô tả
            description_element = soup.find('div', attrs={'data-automation': 'attractionsAboutContent'})
            if description_element:
                description_span = description_element.find('span', class_='JguWG')
                if description_span:
                    description = re.sub(r'\s+', ' ', description_span.text).strip()
                else:
                    description = np.nan
            else:
                description = np.nan
            #time.sleep(random.uniform(0.2, 0.28))

            # Lấy thông tin thời gian tham quan
            duration_element = soup.select_one('div._c > div.biGQs._P.pZUbB.KxBGd')
            if duration_element:
                duration = duration_element.text.strip()
                duration = duration.replace('Duration:', '').strip()
            else:
                duration = np.nan
            #time.sleep(random.uniform(0.2, 0.28))
            
            # Tìm thông tin ranking
            try:
                ranking = driver.find_element(By.CSS_SELECTOR, 'span[class="Wb"]').text
            except NoSuchElementException:
                ranking = np.nan

            # Lấy thông tin giá vé
            try:
                price = driver.find_element(By.CSS_SELECTOR, '.biGQs._P.fiohW.iktvT.uuBRH').text
            except NoSuchElementException:
                price = np.nan

            # Lấy thông tin giờ mở cửa
            try:
                opening_hour = driver.find_element(By.CSS_SELECTOR, '.EFKKt').text
            except NoSuchElementException:
                opening_hour = np.nan

                        
            # Di chuyển chuột đến giữa trang
            ActionChains(driver).move_to_element(driver.find_element(By.TAG_NAME, 'body')).perform()

            # Cuộn xuống cuối trang
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            
            
            # Lấy địa chỉ
            address = soup.select_one('div.wgNTK > div.MJ > button.UikNM._G.B-._S._W._T.c.G_.wSSLS.wnNQG.raEkE > span.biGQs._P.XWJSj.Wb')
            if address:
                address_text = address.text.strip() 
            else:
                address_text = np.nan
            #time.sleep(random.uniform(0.2, 0.28))

            # Lấy website
            try:
                website_elements = driver.find_elements(By.XPATH, "//a[span[text()='Visit website']]")
                
                website_url = np.nan
                for element in website_elements:
                    url = element.get_attribute('href')
                    if "tripadvisor" not in url:  # Bỏ qua link không mong muốn
                        website_url = url
                        break  
            except:
                website_url = np.nan
            #time.sleep(random.uniform(0.2, 0.28))

            # Lấy số điện thoại
            try:
                phone_element = driver.find_element(By.XPATH, "//a[contains(@href, 'tel:')]")
                phone_number = phone_element.get_attribute('href').replace('tel:', '')
            except:
                phone_number = np.nan
            #time.sleep(random.uniform(0.2, 0.28))

            # Lấy email
            try:
                email_element = driver.find_element(By.XPATH, "//a[contains(@href, 'mailto:')]")
                email = email_element.get_attribute('href').replace('mailto:', '')
            except:
                email = np.nan
            #time.sleep(random.uniform(0.2, 0.28))

            # Lấy hình ảnh
            try:
                image_element = driver.find_element(By.CSS_SELECTOR, 'button.pWJvw.w._Z.G_.C-.B-.y._S img')
                image_url = image_element.get_attribute('src')
            except Exception as e:
                image_url = np.nan
            #time.sleep(random.uniform(0.2, 0.28))

            # Lấy số lượng đánh giá theo mức độ
            try:
                excellent_count = int(soup.find("div", string="Excellent").find_next("div", class_="biGQs _P fiohW biKBZ osNWb").get_text().replace(",", ""))
                very_good_count = int(soup.find("div", string="Very good").find_next("div", class_="biGQs _P fiohW biKBZ osNWb").get_text().replace(",", ""))
                average_count = int(soup.find("div", string="Average").find_next("div", class_="biGQs _P fiohW biKBZ osNWb").get_text().replace(",", ""))
                poor_count = int(soup.find("div", string="Poor").find_next("div", class_="biGQs _P fiohW biKBZ osNWb").get_text().replace(",", ""))
                terrible_count = int(soup.find("div", string="Terrible").find_next("div", class_="biGQs _P fiohW biKBZ osNWb").get_text().replace(",", ""))
            except:
                excellent_count = very_good_count = average_count = poor_count = terrible_count = np.nan
                
            #time.sleep(random.uniform(0.2, 0.28))
                        
                        
            near_restaurants = np.nan
            near_attractions = np.nan

            for div in soup.find_all('div', class_='C'):
                near_infor = div.find('div', class_='biGQs _P pZUbB KxBGd')
                if near_infor:
                    near_infor_text = near_infor.text.strip()
                    if "Restaurants" in near_infor_text:
                        restaurant_info = div.find('div', class_='biGQs _P pZUbB mowmC KxBGd')
                        near_restaurants = restaurant_info.text.split()[0] if restaurant_info and restaurant_info.text else np.nan
                    elif "Attractions" in near_infor_text:
                        attraction_info = div.find('div', class_='biGQs _P pZUbB mowmC KxBGd')
                        near_attractions = attraction_info.text.split()[0] if attraction_info and attraction_info.text else np.nan
            
            # Tạo Attraction ID
            attraction_id = f'AT{str(attraction_id_counter).zfill(6)}'
            attraction_id_counter += 1
            #time.sleep(random.uniform(0.2, 0.28))
            

            # Ghi dữ liệu vào CSV
            writer.writerow({
                'Attraction ID': attraction_id,
                'Country': country,
                'Tourist Attraction': place,
                'Name': name,
                'Address': address_text,
                'Website': website_url,
                'Phone Number': phone_number,
                'Category': category,
                'Ranking': ranking,
                'Opening Hour': opening_hour,
                'Description': description,
                'Duration': duration,
                'Price': price,
                'Email': email,
                'Rating': rating_value,
                'Review Count': review_count,
                'Excellent Reviews': excellent_count,
                'Very Good Reviews': very_good_count,
                'Average Reviews': average_count,
                'Poor Reviews': poor_count,
                'Terrible Reviews': terrible_count,
                'Near Restaurants': near_restaurants,
                'Near Attractions': near_attractions,
                "URL": attraction_url,
                'Image URL': image_url
            })
            
            # Cập nhật thanh tiến độ sau mỗi địa điểm
            pbar.update(1)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".biGQs._P.fiohW.eIegw"}
  (Session info: chrome=136.0.7103.93); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x003FFC83+61635]
	GetHandleVerifier [0x003FFCC4+61700]
	(No symbol) [0x002205D3]
	(No symbol) [0x0026899E]
	(No symbol) [0x00268D3B]
	(No symbol) [0x002B0E12]
	(No symbol) [0x0028D2E4]
	(No symbol) [0x002AE61B]
	(No symbol) [0x0028D096]
	(No symbol) [0x0025C840]
	(No symbol) [0x0025D6A4]
	GetHandleVerifier [0x006845A3+2701795]
	GetHandleVerifier [0x0067FD26+2683238]
	GetHandleVerifier [0x0069AA6E+2793134]
	GetHandleVerifier [0x00416945+155013]
	GetHandleVerifier [0x0041D02D+181357]
	GetHandleVerifier [0x004074D8+92440]
	GetHandleVerifier [0x00407680+92864]
	GetHandleVerifier [0x003F2070+5296]
	BaseThreadInitThunk [0x752E7BA9+25]
	RtlInit